In [8]:
# Platform configs
url = "http://localhost:4000"
project_id = "65df5715dbfd389a9619eabb"

In [40]:
# Model setup
checkpoint = "openai-community/gpt2"  # embeddings are "transformer.wte"

device = "cpu"

# Model args
model_args = {
    "output_attentions": True,
    # You could change the temperature, but not that then creating different kinds of explanations does not work.
    "temperature": 1, # Note that if you add this, then you need to add "do_sample": True
    "do_sample": True
}

final_config = {
    "platform_url": url,
    "project_id": project_id,
    "model_type": "dec",
    "embeddings": "transformer.wte",
}


In [41]:
from llmex import from_pretrained

In [42]:
model = from_pretrained(checkpoint, config=final_config, model_args=model_args, device=device)

setting up model
post init
model reset


In [43]:
# “Imagination is more important than knowledge. For knowledge is limited to all we now know and understand, while imagination embraces the entire world, and all there ever will be to know and understand.”
prompt = "Quote: Imagination is more"

In [48]:
result = model.predict(prompt, generate_explanations=False)
result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


updated app


' difficult when you are not making a profit from your'

In [49]:
run_id = "66069e516455702029922429"
res = model.predict_from_run(run_id, explanation_type="saliency")

wrong value type for field "output_alternatives" - should be "list" instead of value "[[{'token': 'Ġthan', 'score': -102.0221939086914}, {'token': 'Ġimportant', 'score': -102.17761993408203}, {'token': 'Ġpowerful', 'score': -103.70684051513672}, {'token': 'Ġof', 'score': -104.01897430419922}, {'token': 'Ġabout', 'score': -104.02540588378906}], [{'token': 'Ġto', 'score': -86.72744750976562}, {'token': 'Ġthan', 'score': -87.28730773925781}, {'token': '.', 'score': -88.95866394042969}, {'token': 'Ġfor', 'score': -89.04570770263672}, {'token': ',', 'score': -89.0953369140625}], [{'token': 'Ġyou', 'score': -90.63545227050781}, {'token': 'Ġit', 'score': -92.01100158691406}, {'token': 'Ġthe', 'score': -92.38310241699219}, {'token': 'Ġwe', 'score': -92.59398651123047}, {'token': 'Ġthere', 'score': -93.20353698730469}], [{'token': "'re", 'score': -129.20887756347656}, {'token': 'Ġhave', 'score': -129.83990478515625}, {'token': 'Ġare', 'score': -129.99484252929688}, {'token': 'Ġdon', 'score': -1

ValueError: Run with id 66069e516455702029922429 not found

In [8]:
res

'Dave lives in Palm Coast, FL and is a lawyer. His personal interests include the law, politics,'